<a href="https://colab.research.google.com/github/xwang222/My-Projects/blob/main/Neural_Network_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All the dataframes are created for demonstration in Excel. They are not necessary for a neural network to work.

In [ ]:
from sklearn.datasets import fetch_openml
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
import time
import pandas as pd

In [ ]:
x_train=pd.read_csv('/content/drive/MyDrive/ML Scratch/x_train.csv', header = None)  
y_train_df=pd.read_csv('/content/drive/MyDrive/ML Scratch/y_train.csv', header = None)  

In [ ]:
sizes=[7, 5, 4, 3]
input_layer=sizes[0]
hidden_1=sizes[1]
hidden_2=sizes[2]
output_layer=sizes[3]

params = {
    'W1':np.random.randn(hidden_1, input_layer) * np.sqrt(1. / hidden_1),
    'W2':np.random.randn(hidden_2, hidden_1) * np.sqrt(1. / hidden_2),
    'W3':np.random.randn(output_layer, hidden_2) * np.sqrt(1. / output_layer)
}

def sigmoid(x, derivative=False):
        if derivative:
            return (np.exp(-x))/((np.exp(-x)+1)**2)
        return 1/(1 + np.exp(-x))

def softmax(x, derivative=False):
    # Numerically stable with large exponentials
    exps = np.exp(x - x.max())
    if derivative:
        return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
    return exps / np.sum(exps, axis=0)

w1_all_df=pd.DataFrame()
w2_all_df=pd.DataFrame()
w3_all_df=pd.DataFrame()
A1_all_df=pd.DataFrame()
A2_all_df=pd.DataFrame()
A3_all_df=pd.DataFrame()
z1_all_df=pd.DataFrame()
z2_all_df=pd.DataFrame()
z3_all_df=pd.DataFrame()
ch1_all_df=pd.DataFrame()
ch2_all_df=pd.DataFrame()
ch3_all_df=pd.DataFrame()
w1_new_all_df=pd.DataFrame()
w2_new_all_df=pd.DataFrame()
w3_new_all_df=pd.DataFrame()
err1_all_df=pd.DataFrame()
err2_all_df=pd.DataFrame()
err3_all_df=pd.DataFrame()

In [ ]:
for j in range(1):
  for i in range(len(x_train)):
    x_dm=x_train.iloc[[i]]
    y_dm=y_train_df.iloc[[i]]

    # input layer activations becomes sample
    params['A0'] = x_dm
    A0_df=pd.DataFrame(params["A0"])

    # input layer to hidden layer 1
    params['Z1'] = np.dot(params["W1"], params['A0'].T)
    params['A1'] = sigmoid(params['Z1'])

    w1_df=pd.DataFrame(params["W1"])
    z1_df=pd.DataFrame(params["Z1"])
    A1_df=pd.DataFrame(params["A1"])

    w1_all_df=w1_all_df.append(w1_df, ignore_index=True)
    z1_all_df=z1_all_df.append(z1_df, ignore_index=True)
    A1_all_df=A1_all_df.append(A1_df, ignore_index=True)

    params['Z2'] = np.dot(params["W2"], params['A1'])
    params['A2'] = sigmoid(params['Z2'])

    w2_df=pd.DataFrame(params["W2"])
    z2_df=pd.DataFrame(params["Z2"])
    A2_df=pd.DataFrame(params["A2"])

    w2_all_df=w2_all_df.append(w2_df, ignore_index=True)
    z2_all_df=z2_all_df.append(z2_df, ignore_index=True)
    A2_all_df=A2_all_df.append(A2_df, ignore_index=True)

    params['Z3'] = np.dot(params["W3"], params['A2'])
    params['A3'] = softmax(params['Z3'])

    w3_df=pd.DataFrame(params["W3"])
    z3_df=pd.DataFrame(params["Z3"])
    A3_df=pd.DataFrame(params["A3"])

    w3_all_df=w3_all_df.append(w3_df, ignore_index=True)
    z3_all_df=z3_all_df.append(z3_df, ignore_index=True)
    A3_all_df=A3_all_df.append(A3_df, ignore_index=True)

    change_w = {}
    output = params['A3']
    # Calculate W3 update
    error = 2 * (output.T - y_dm) / output.shape[1] * softmax(params['Z3'], derivative=True).T
    change_w['W3'] = np.outer(error.T, params['A2'])

    ch3_df=pd.DataFrame(change_w["W3"])
    ch3_all_df=ch3_all_df.append(ch3_df, ignore_index=True)
    err3_df = pd.DataFrame(error)
    err3_all_df=err3_all_df.append(err3_df, ignore_index=True)

    # Calculate W2 update
    error = np.dot(error,params['W3']) * sigmoid(params['Z2'], derivative=True).T
    change_w['W2'] = np.outer(error.T, params['A1'])

    ch2_df=pd.DataFrame(change_w["W2"])
    ch2_all_df=ch2_all_df.append(ch2_df, ignore_index=True)
    err2_df = pd.DataFrame(error)
    err2_all_df=err2_all_df.append(err2_df, ignore_index=True)

    # Calculate W1 update
    error = np.dot(error,params['W2']) * sigmoid(params['Z1'], derivative=True).T
    change_w['W1'] = np.outer(error.T, params['A0'])

    ch1_df=pd.DataFrame(change_w["W1"])
    ch1_all_df=ch1_all_df.append(ch1_df, ignore_index=True)
    err1_df = pd.DataFrame(error)
    err1_all_df=err1_all_df.append(err1_df, ignore_index=True)    

    l_rate = 1
    for key, value in change_w.items():
              params[key] -= l_rate * value

    w1_df_new=pd.DataFrame(params["W1"])
    w2_df_new=pd.DataFrame(params["W2"])
    w3_df_new=pd.DataFrame(params["W3"])

    w1_new_all_df=w1_new_all_df.append(w1_df_new, ignore_index=True)
    w2_new_all_df=w2_new_all_df.append(w2_df_new, ignore_index=True)
    w3_new_all_df=w3_new_all_df.append(w3_df_new, ignore_index=True)

In [ ]:
w1_all_df.to_csv('w1.csv')
w2_all_df.to_csv('w2.csv')
w3_all_df.to_csv('w3.csv')
A1_all_df.to_csv('A1.csv')
A2_all_df.to_csv('A2.csv')
A3_all_df.to_csv('A3.csv')
z1_all_df.to_csv('Z1.csv')
z2_all_df.to_csv('Z2.csv')
z3_all_df.to_csv('Z3.csv')
ch1_all_df.to_csv('ch1.csv')
ch2_all_df.to_csv('ch2.csv')
ch3_all_df.to_csv('ch3.csv')
w1_new_all_df.to_csv('w1_new.csv')
w2_new_all_df.to_csv('w2_new.csv')
w3_new_all_df.to_csv('w3_new.csv')
err1_all_df.to_csv('err1_new.csv')
err2_all_df.to_csv('err2_new.csv')
err3_all_df.to_csv('err3_new.csv')